In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import random
import os
import pandas as pd


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

os.chdir("/home2/s439906/project/CASP16/monomer/")
path = "./monomer_data_newest/processed/"
files = [file for file in os.listdir(path) if file.startswith("T") and file.endswith(".csv")]
files.sort()

df = pd.DataFrame()
for file in files:
    df_tmp = pd.read_csv(path + file, index_col=0)
    df = pd.concat([df, df_tmp])
df.index = df.index.str.extract(
    r'(T\w+)TS(\w+)_(\w+)(?:-(.*))?').apply(lambda x: (f"{x[0]}-{x[3]}" if pd.notna(x[3]) else x[0], f"TS{x[1]}", x[2]), axis=1)

df = pd.DataFrame()
for file in files:
    df_tmp = pd.read_csv(path + file, index_col=0)
    df = pd.concat([df, df_tmp])
df.index = df.index.str.extract(
    r'(T\w+)TS(\w+)_(\w+)(?:-(.*))?').apply(lambda x: (f"TS{x[1]}", f"{x[0]}-{x[3]}" if pd.notna(x[3]) else x[0],  x[2]), axis=1)
# sort index
df = df.sort_index()
df.index = pd.MultiIndex.from_tuples(
            df.index, names=['group', 'target', 'model'])



# 创建一个空的 DataFrame
df = pd.DataFrame()

# 遍历所有文件并读取内容，然后合并到一个 DataFrame 中
for file in files:
    df_tmp = pd.read_csv(os.path.join(path, file), index_col=0)
    df = pd.concat([df, df_tmp])

# 提取并转换索引
df.index = df.index.str.extract(
    r'(T\w+)TS(\w+)_(\w+)(?:-(.*))?'
).apply(lambda x: (f"TS{x[1]}", f"{x[0]}-{x[3]}" if pd.notna(x[3]) else x[0], x[2]), axis=1)

# 排序索引
df = df.sort_index()

# 创建 MultiIndex 并指定层级名称
df.index = pd.MultiIndex.from_tuples(
    df.index, names=['group', 'target', 'model']
)

# 检查每个 group 的 target 数目
group_target_counts = df.index.get_level_values('group').value_counts()
print(group_target_counts)




In [ ]:
# group_dict = {}
# for file in files:
#     EU = file.split('.')[0]
#     with open(path + file, 'r') as f:
#         for line in f:
#             if line.startswith("T"):
#                 word = line.strip().split(',')
#                 meta_data = word[0]
#                 _, meta_data = meta_data.split('TS')
#                 group = meta_data.split('_')[0]
#                 group = 'TS' + group
#                 if group not in group_dict:
#                     group_dict[group] = {}
#                 if EU not in group_dict[group]:
#                     group_dict[group][EU] = []
#                 group_dict[group][EU].append(word[1:])
# to_del = []
# # print the groups that has less than 37 targets
# for group in group_dict:
#     if len(group_dict[group]) <= 70:
#         # print(group, len(group_dict[group]))
#         to_del.append(group)
# for group in to_del:
#     del group_dict[group]

# group_dict = dict(sorted(group_dict.items(), key=lambda x: x[0]))
# weights = [
#                         1/6, 1/16, 1/6,
#                         1/6,
#                         1/16, 1/8,
#                         1/8, 1/16,
#                         1/16,
#                     ]
# # get a n*n matrix to store bootstrap results
# # n = len(group_dict)
# # bootstrap_results = np.zeros((n, n))
# bootstrap_dict  = {}
# for i in range(2):
#     for group_1, group_2 in combinations(group_dict.keys(), 2):
#         data_1 = group_dict[group_1]
#         data_2 = group_dict[group_2]
#         # 获取共有的 EU
#         common_EU = set(data_1.keys()) & set(data_2.keys())
#         # 随机有放回地抽取 len(common_EU) 个 EU
#         sampled_EU = random.choices(list(common_EU), k=len(common_EU))
#         score_1 = 0
#         score_2 = 0
#         # 对于每个抽取的 EU，取出对应的值，然后从值的列表中随机抽取一个
#         for eu in sampled_EU:
#             value_list_1 = data_1[eu]
#             value_list_2 = data_2[eu]
#             sampled_value_1 = random.choice(value_list_1)
#             sampled_value_2 = random.choice(value_list_2)
#             weighed_sum_1 = np.sum([float(x) * y for x, y in zip(sampled_value_1, weights)])
#             weighed_sum_2 = np.sum([float(x) * y for x, y in zip(sampled_value_2, weights)])
#             score_1 += float(weighed_sum_1)
#             score_2 += float(weighed_sum_2)
#         if score_1 > score_2:
#             if group_1 not in bootstrap_dict:
#                 bootstrap_dict[group_1] = {}
#             if group_2 not in bootstrap_dict[group_1]:
#                 bootstrap_dict[group_1][group_2] = 0
#             bootstrap_dict[group_1][group_2] += 1
#         elif score_1 < score_2:
#             if group_2 not in bootstrap_dict:
#                 bootstrap_dict[group_2] = {}
#             if group_1 not in bootstrap_dict[group_2]:
#                 bootstrap_dict[group_2][group_1] = 0
#             bootstrap_dict[group_2][group_1] += 1
#         else:
#             if group_1 not in bootstrap_dict:
#                 bootstrap_dict[group_1] = {}
#             if group_2 not in bootstrap_dict[group_1]:
#                 bootstrap_dict[group_1][group_2] = 0
#             bootstrap_dict[group_1][group_2] += 0.5
#             if group_2 not in bootstrap_dict:
#                 bootstrap_dict[group_2] = {}
#             if group_1 not in bootstrap_dict[group_2]:
#                 bootstrap_dict[group_2][group_1] = 0
#             bootstrap_dict[group_2][group_1] += 0.5


#     print(f"Bootstrap {i+1} done.")

# bootstrap_sum = {}
# for group_1 in bootstrap_dict:
#     for group_2 in bootstrap_dict[group_1]:
#         if group_1 not in bootstrap_sum:
#             bootstrap_sum[group_1] = 0
#         bootstrap_sum[group_1] += bootstrap_dict[group_1][group_2]
# # sort the dict by value
# bootstrap_sum = dict(sorted(bootstrap_sum.items(), key=lambda x: x[1], reverse=True))
# bootstrap_sum

In [ ]:
weights = [
                        1/6, 1/16, 1/6,
                        1/6,
                        1/16, 1/8,
                        1/8, 1/16,
                        1/16,
                    ]
group_dict = {}
for file in files:
    EU = file.split('.')[0]
    with open(path + file, 'r') as f:
        for line in f:
            if line.startswith("T"):
                word = line.strip().split(',')
                meta_data = word[0]
                _, meta_data = meta_data.split('TS')
                group = meta_data.split('_')[0]
                group = 'TS' + group
                if group not in group_dict:
                    group_dict[group] = {}
                if EU not in group_dict[group]:
                    group_dict[group][EU] = []
                # print(word)
                # print([word[i+1] * weights[i] for i in range(9)])
                group_dict[group][EU].append(sum([float(word[i+1] )* weights[i] for i in range(9)]))

to_del = []
# print the groups that has less than 37 targets
for group in group_dict:
    if len(group_dict[group]) <= 107:
        # print(group, len(group_dict[group]))
        to_del.append(group)
for group in to_del:
    del group_dict[group]
group_dict = dict(sorted(group_dict.items(), key=lambda x: x[0]))
group_dict

In [ ]:
# get a n*n matrix to store bootstrap results
# n = len(group_dict)
# bootstrap_results = np.zeros((n, n))
bootstrap_dict  = {}

for i in range(100):
    for group_1, group_2 in combinations(group_dict.keys(), 2):
        data_1 = group_dict[group_1]
        data_2 = group_dict[group_2]
        common_EU = set(data_1.keys()) & set(data_2.keys())
        sampled_EU = random.choices(list(common_EU), k=len(common_EU))
        score_1 = 0
        score_2 = 0
        for eu in sampled_EU:
            value_list_1 = data_1[eu]
            value_list_2 = data_2[eu]
            # sampled_value_1 = random.choice(value_list_1)
            # sampled_value_2 = random.choice(value_list_2)
            # sampled_value_1 = max(value_list_1)
            # sampled_value_2 = max(value_list_2)

            if len(value_list_1) != len(value_list_2):
                if len(value_list_1) > len(value_list_2):
                    # first make sample list 1 to be the same length as sample list 2 without putting back
                    value_list_1_tmp = random.sample(value_list_1, len(value_list_2))
                    sampled_value_1 = max(value_list_1_tmp)
                    sampled_value_2 = max(value_list_2)
                else:
                    value_list_2_tmp = random.sample(value_list_2, len(value_list_1))
                    sampled_value_1 = max(value_list_1)
                    sampled_value_2 = max(value_list_2_tmp)
            else:
                sampled_value_1 = max(value_list_1)
                sampled_value_2 = max(value_list_2)
            score_1 += sampled_value_1
            score_2 += sampled_value_2
        if score_1 > score_2:
            if group_1 not in bootstrap_dict:
                bootstrap_dict[group_1] = {}
            if group_2 not in bootstrap_dict[group_1]:
                bootstrap_dict[group_1][group_2] = 0
            bootstrap_dict[group_1][group_2] += 1
        elif score_1 < score_2:
            if group_2 not in bootstrap_dict:
                bootstrap_dict[group_2] = {}
            if group_1 not in bootstrap_dict[group_2]:
                bootstrap_dict[group_2][group_1] = 0
            bootstrap_dict[group_2][group_1] += 1
        else:
            if group_1 not in bootstrap_dict:
                bootstrap_dict[group_1] = {}
            if group_2 not in bootstrap_dict[group_1]:
                bootstrap_dict[group_1][group_2] = 0
            bootstrap_dict[group_1][group_2] += 0.5
            if group_2 not in bootstrap_dict:
                bootstrap_dict[group_2] = {}
            if group_1 not in bootstrap_dict[group_2]:
                bootstrap_dict[group_2][group_1] = 0
            bootstrap_dict[group_2][group_1] += 0.5


    print(f"Bootstrap {i+1} done.")
bootstrap_sum = {}
for group_1 in bootstrap_dict:
    for group_2 in bootstrap_dict[group_1]:
        if group_1 not in bootstrap_sum:
            bootstrap_sum[group_1] = 0
        bootstrap_sum[group_1] += bootstrap_dict[group_1][group_2]
# sort the dict by value
bootstrap_sum = dict(sorted(bootstrap_sum.items(), key=lambda x: x[1], reverse=True))
bootstrap_sum

In [ ]:
weights = [
                        1/6, 1/16, 1/6,
                        1/6,
                        1/16, 1/8,
                        1/8, 1/16,
                        1/16,
                    ]
group_dict = {}
for file in files:
    EU = file.split('.')[0]
    with open(path + file, 'r') as f:
        for line in f:
            if line.startswith("T"):
                word = line.strip().split(',')
                meta_data = word[0]
                _, meta_data = meta_data.split('TS')
                group, model_id = meta_data.split('_')
                group = 'TS' + group
                model_id = model_id[0]
                if group not in group_dict:
                    group_dict[group] = {}
                if EU not in group_dict[group]:
                    group_dict[group][EU] = {}
                score = sum([float(word[i+1] )* weights[i] for i in range(9)])
                if model_id not in group_dict[group][EU]:
                    group_dict[group][EU][model_id] = score
                else:
                    group_dict[group][EU][model_id] = max(score, group_dict[group][EU][model_id])
    # if there is no model_1, replace the model with the smallest id to be model_1
    for group in group_dict:
        for EU in group_dict[group]:
            if '1' not in group_dict[group][EU]:
                min_model = min(group_dict[group][EU], key=group_dict[group][EU].get)
                group_dict[group][EU]['1'] = group_dict[group][EU][min_model]
                del group_dict[group][EU][min_model]
 
to_del = []
# print the groups that has less than 37 targets
for group in group_dict:
    if len(group_dict[group]) <= 55:
        print(group, len(group_dict[group]))
        to_del.append(group)
for group in to_del:
    del group_dict[group]
group_dict = dict(sorted(group_dict.items(), key=lambda x: x[0]))

# get a n*n matrix to store bootstrap results
# n = len(group_dict)
# bootstrap_results = np.zeros((n, n))
bootstrap_dict  = {}

for i in range(50):
    for group_1, group_2 in combinations(group_dict.keys(), 2):
        data_1 = group_dict[group_1]
        data_2 = group_dict[group_2]
        common_EU = set(data_1.keys()) & set(data_2.keys())
        sampled_EU = random.choices(list(common_EU), k=len(common_EU))
        score_1 = 0
        score_2 = 0
        for eu in sampled_EU:
            value_list_1 = data_1[eu]
            value_list_2 = data_2[eu]
            # sampled_value_1 = random.choice(value_list_1)
            # sampled_value_2 = random.choice(value_list_2)
            # sampled_value_1 =  max(value_list_1.values())
            # sampled_value_2 =  max(value_list_2.values())
            sampled_value_1 = value_list_1['1']
            sampled_value_2 = value_list_2['1']
            score_1 += sampled_value_1
            score_2 += sampled_value_2
        if score_1 > score_2:
            if group_1 not in bootstrap_dict:
                bootstrap_dict[group_1] = {}
            if group_2 not in bootstrap_dict[group_1]:
                bootstrap_dict[group_1][group_2] = 0
            bootstrap_dict[group_1][group_2] += 1
        elif score_1 < score_2:
            if group_2 not in bootstrap_dict:
                bootstrap_dict[group_2] = {}
            if group_1 not in bootstrap_dict[group_2]:
                bootstrap_dict[group_2][group_1] = 0
            bootstrap_dict[group_2][group_1] += 1
        else:
            if group_1 not in bootstrap_dict:
                bootstrap_dict[group_1] = {}
            if group_2 not in bootstrap_dict[group_1]:
                bootstrap_dict[group_1][group_2] = 0
            bootstrap_dict[group_1][group_2] += 0.5
            if group_2 not in bootstrap_dict:
                bootstrap_dict[group_2] = {}
            if group_1 not in bootstrap_dict[group_2]:
                bootstrap_dict[group_2][group_1] = 0
            bootstrap_dict[group_2][group_1] += 0.5


    print(f"Bootstrap {i+1} done.")
bootstrap_sum = {}
for group_1 in bootstrap_dict:
    for group_2 in bootstrap_dict[group_1]:
        if group_1 not in bootstrap_sum:
            bootstrap_sum[group_1] = 0
        bootstrap_sum[group_1] += bootstrap_dict[group_1][group_2]
# sort the dict by value
bootstrap_sum = dict(sorted(bootstrap_sum.items(), key=lambda x: x[1], reverse=True))
bootstrap_sum

In [ ]:
group_points = {}
for group in group_dict:
    for EU in group_dict[group]:
        if group not in group_points:
            group_points[group] = 0
        print(group_dict[group][EU].__len__())
        group_points[group] += max(group_dict[group][EU])
        break
    break
group_points = dict(sorted(group_points.items(), key=lambda x: x[1], reverse=True))
group_points